# 5)	Download data from: http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3.csv

These data list passengers on the Titanic and whether they survived the sinking of the ship.
a. 	 Import the data to SKLearn, retaining the following data fields: pclass, survived (the target variable), sex, age, sibsp, parch, fare, embarked. (descriptions of these fields are provided here: http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/Ctitanic3.html#pclass ). Segment the data by those who survived and those who did not survive. For those who survived, indicate the mean of each continuous variable and the mode of each categorical variable.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import statsmodels as sm
import os

In [4]:
path = "/content/drive/My Drive/EMSE6575/titanic3.csv"
df = pd.read_csv(path)

In [5]:
df = df[['pclass', 'survived', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']]
survived = df[df.survived==1]
not_survived = df[df.survived==0]

In [10]:
def mean_mode(df, use_col):
    for key, value in df.select_dtypes(np.number).mean().items():
        if key not in [use_col, 'pclass']:
            print(f'Mean for {key} is {value}.')
    print('\n')
    if use_col != 'pclass':
        print(f'Mode for passenger class is {df.pclass.mode().item()}.')
    for col in df.select_dtypes(object).columns:
        if col != use_col:
            print(f'Mode for {col} is {df[col].mode().item()}.')


In [12]:
print('Number of Survived: \n')
mean_mode(survived, 'survived')

Number of Survived: 

Mean for age is 28.91824355971897.
Mean for sibsp is 0.462.
Mean for parch is 0.476.
Mean for fare is 49.361183600000004.


Mode for passenger class is 1.
Mode for sex is female.
Mode for embarked is S.


In [13]:
df.groupby('survived').agg({'age': 'mean', 
                          'sibsp': 'mean', 
                          'parch': 'mean', 
                          'fare': 'mean',
                          'sex': pd.Series.mode,
                          'pclass': pd.Series.mode,
                          'embarked': pd.Series.mode})

,age,sibsp,parch,fare,sex,pclass,embarked
survived,,,,,,,
0,30.545363,0.521632,0.328801,23.353831,male,3,S
1,28.918244,0.462000,0.476000,49.361184,female,1,S


b. 	What is the mean of each continuous variable and the mode of each categorical variable for each sex? What is the observed probability of survival for each sex?



In [15]:
female_df = df[df.sex=='female']
male_df = df[df.sex=='male']

print('For female passengers: \n')
mean_mode(female_df, 'sex')
print('For male passengers: \n')
mean_mode(male_df, 'sex')

For female passengers: 

Mean for survived is 0.7274678111587983.
Mean for age is 28.68708762886598.
Mean for sibsp is 0.6523605150214592.
Mean for parch is 0.6330472103004292.
Mean for fare is 46.19809656652361.


Mode for passenger class is 3.
Mode for embarked is S.
For male passengers: 

Mean for survived is 0.19098457888493475.
Mean for age is 30.58522796352584.
Mean for sibsp is 0.4139976275207592.
Mean for parch is 0.2479240806642942.
Mean for fare is 26.154600831353918.


Mode for passenger class is 3.
Mode for embarked is S.


In [16]:
df.groupby('sex').agg({'survived': 'mean', 
                          'age': 'mean', 
                          'sibsp': 'mean', 
                          'parch': 'mean', 
                          'fare': 'mean',
                          'pclass': pd.Series.mode,
                          'embarked': pd.Series.mode})

,survived,age,sibsp,parch,fare,pclass,embarked
sex,,,,,,,
female,0.727468,28.687088,0.652361,0.633047,46.198097,3,S
male,0.190985,30.585228,0.413998,0.247924,26.154601,3,S


Probability of survived of female is higher than male.

c. 	What is the mean of each continuous variable and the mode of each categorical variable for each passenger class? What is the observed probability of survival for each passenger class?


In [22]:
class_1 = df[df.pclass==1]
class_2 = df[df.pclass==2]
class_3 = df[df.pclass==3]

print('Class 1: \n')
mean_mode(class_1, 'pclass')

print('Class 2: \n')
mean_mode(class_2, 'pclass')

print('Class 3: \n')
mean_mode(class_3, 'pclass')

Class 1: 

Mean for survived is 0.6191950464396285.
Mean for age is 39.15992957746479.
Mean for sibsp is 0.43653250773993807.
Mean for parch is 0.3653250773993808.
Mean for fare is 87.50899164086688.


Mode for sex is male.
Mode for embarked is S.
Class 2: 

Mean for survived is 0.4296028880866426.
Mean for age is 29.506704980842912.
Mean for sibsp is 0.3935018050541516.
Mean for parch is 0.36823104693140796.
Mean for fare is 21.179196389891697.


Mode for sex is male.
Mode for embarked is S.
Class 3: 

Mean for survived is 0.2552891396332863.
Mean for age is 24.81636726546906.
Mean for sibsp is 0.5684062059238364.
Mean for parch is 0.4005641748942172.
Mean for fare is 13.302888700564973.


Mode for sex is male.
Mode for embarked is S.


In [18]:
df.groupby('pclass').agg({'survived': 'mean', 
                          'age': 'mean', 
                          'sibsp': 'mean', 
                          'parch': 'mean', 
                          'fare': 'mean',
                          'sex': pd.Series.mode,
                          'embarked': pd.Series.mode})

,survived,age,sibsp,parch,fare,sex,embarked
pclass,,,,,,,
1,0.619195,39.159930,0.436533,0.365325,87.508992,male,S
2,0.429603,29.506705,0.393502,0.368231,21.179196,male,S
3,0.255289,24.816367,0.568406,0.400564,13.302889,male,S


d. 	Fit a logistic regression model to the data using all variables from part 1. Using 10-fold cross-validation, what is the model’s average prediction accuracy?


In [21]:
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

In [20]:
df.isnull().sum()

pclass        0
survived      0
sex           0
age         263
sibsp         0
parch         0
fare          1
embarked      2
dtype: int64

In [29]:
df['pclass'] = df['pclass'].astype(str)
df_encoded = pd.concat([df.drop(['pclass', 'sex', 'embarked'], axis=1), 
           pd.get_dummies(df[['pclass', 'sex', 'embarked']], drop_first=True)], axis=1)
X, y = df_encoded.drop('survived', axis=1), df['survived']

In [24]:
imp = SimpleImputer(strategy='median')
X = pd.DataFrame(imp.fit_transform(X), columns=X.columns)

kf = KFold(n_splits=10)
accuracy = 0
for train, test in kf.split(X):
    lr = LogisticRegression(random_state=1).fit(X.iloc[train], y.iloc[train])
    test_res = lr.predict(X.iloc[test])
    accuracy += accuracy_score(y.iloc[test], test_res)
print('10 fold cross validation accuracy is ', accuracy/10)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

10 fold cross validation accuracy is  0.7845860246623606


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

e. 	Now that you’ve fit a model using logistic regression, calculate the AIC and BIC of this model on all of the data.


In [36]:
import itertools 
from sklearn.metrics import log_loss

In [37]:
n_features = len(X)
min_aic = np.inf
min_bic = np.inf

best_features = []
best_model = None

for comb in range(1, len(df_encoded.columns)-1):
    for combination in itertools.combinations(list(range(1, len(df_encoded.columns)-1)), comb):
        
        X = df_encoded.drop('survived', axis=1)
        X = X.iloc[:, (k for k in combination)]
        
        imp = SimpleImputer(strategy='median')
        X = pd.DataFrame(imp.fit_transform(X), columns=X.columns)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
        result = LogisticRegression(random_state=1).fit(X_train, y_train)
        y_pred = result.predict(X_test)
        
        logloss = log_loss(y_test, y_pred)
        n = len(X)
        p = len(X.columns)
        aic = 2*p/n - 2/n*logloss
        bic = np.log(n)*p - 2*logloss
        
        if aic<min_aic or bic<min_bic:
            min_aic = aic
            min_bic = bic
            best_features = X.columns
            best_model = result
            

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

In [39]:
print("Min AIC : ",min_aic, " Min BIC : ", min_bic)
print(best_features)

Min AIC :  -0.02369525368525626  Min BIC :  -25.840068308090544
Index(['parch'], dtype='object')


f.  	Using only these measures, what is a model that is more parsimonious according to both AIC and BIC?


Model 1 that includes only the number of siblings/spouses on board which is more parsimonious.

g. 	What is the accuracy of this model?


In [41]:
X, y = df_encoded.drop('survived', axis=1), df['survived']
acc = best_model.score(X[best_features], y)
print(f'Accuracy of the model: {round(acc*100, 2)}%.')

Accuracy of the model: 60.89%.


h. 	How does it compare to a random coin flip?

Random coin flip results in 50% accuracy, while the model has around 61% accuracy. Therefore, it is 11% better. The experiment below is illustrating this. As you can see, when we choose values in random, we get accuracy close to 50%. But our model had 61% accuracy.


In [43]:
rand_preds = np.random.randint(0, 2, len(X))
print('Random predictions accuracy: ', accuracy_score(y, rand_preds))

Random predictions accuracy:  0.5110771581359816


i.  	How does it compare to a model that constantly predicts one outcome?


In [44]:
one_preds = [1] * len(X)
print('One outcome prediction : ', accuracy_score(y, one_preds))

One outcome prediction :  0.3819709702062643


j.  	How does it compare to the “saturated” model that includes all predictors in the dataset?


In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
imp = SimpleImputer(strategy='median')
X_train = pd.DataFrame(imp.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(imp.transform(X_test), columns=X_test.columns)
result = LogisticRegression(random_state=1).fit(X_train, y_train)
acc = result.score(X_test, y_test)
print(f'Accuracy of the model: {round(acc*100, 2)}%.')

Accuracy of the model: 78.24%.


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


k. 	Report the coefficients of the parsimonious model and interpret, in words, what they mean

In [38]:
coefficients = pd.DataFrame({'feature': list(best_features), 'coefficient': best_model.coef_[0]})
coefficients

,feature,coefficient
0,parch,0.189414
